In [5]:
import sys
import gi
import logging
import datetime
from threading import Thread, Event

gi.require_version("GLib", "2.0")
gi.require_version("GObject", "2.0")
gi.require_version("Gst", "1.0")

from gi.repository import Gst, GLib, GObject

uri_name='rtsp://127.0.0.1:8554/test'
file='/home/fkurt/Murat/ElephantsDream.mp4'

log = logging.getLogger('pad_probe')
def logging_pad_probe(pad, probeinfo, location):
    pts_nanpseconds = probeinfo.get_buffer().pts
    pts_timedelta = datetime.timedelta(microseconds=pts_nanpseconds / 1000)
    log.debug("PTS at %s = %s", '{:>20s}'.format(location), pts_timedelta)
    return Gst.PadProbeReturn.OK

def cb_newpad(demux, src, sink):
    print("In cb_newpad\n")
    caps=src.get_current_caps()
    gststruct=caps.get_structure(0)
    gstname=gststruct.get_name()
    features=caps.get_features(0)

    print("gstname=",gstname)
    if(gstname.find("video")!=-1):
        print("features=",features)
        sink_pad=sink.get_static_pad("sink")
        if not src.link(sink_pad):
            sys.stderr.write(f"Failed to link src pad to sink pad:{gstname}\n")


logging.basicConfig(level=logging.DEBUG, format="[%(name)s] [%(levelname)8s] - %(message)s")
logger = logging.getLogger(__name__)

Gst.init(None)

pipeline=Gst.Pipeline.new('test-pipeline')

source=Gst.ElementFactory.make('filesrc','source')
source.set_property('location', file)

demux=Gst.ElementFactory.make("qtdemux", "demux")
if not demux:
    sys.stderr.write(" Unable to create demux \n")
decoder=Gst.ElementFactory.make('avdec_h264','decoder')
demux.connect("pad-added", cb_newpad, decoder)

mixer=Gst.ElementFactory.make('videomixer','mixer')

sink=Gst.ElementFactory.make('xvimagesink','sink')

if not sink or not decoder or not mixer or not pipeline:
    logger.error('Elementlerde sıkıntı var...')
    sys.exit(1)

pipeline.add(source)
pipeline.add(demux)
pipeline.add(decoder)
pipeline.add(mixer)

pipeline.add(sink)

if source.link(demux):
    logger.info('src-demux Bağlandı')
if decoder.link(mixer):
    logger.info('decoder-videoconvert Bağlandı')
if mixer.link(sink):
    logger.info('videoconvert-x264enc Bağlandı')


def add_new_src():
    logger.info("Adding testsrc2")
    testsrc2 = Gst.ElementFactory.make("videotestsrc", "src2")
    testsrc2.set_property("freq", 440)
    testsrc2.set_property("is-live", True)  # (2)

    testsrc2.get_static_pad("src").add_probe(
        Gst.PadProbeType.BUFFER, logging_pad_probe, "src2-output")

    pipeline.add(testsrc2)
    testsrc2.link_filtered(mixer)
    testsrc2.sync_state_with_parent()  # (4)
    logger.info("Adding src2 done")


stop_event = Event()


def timed_sequence():
    log.info("Starting Sequence")
    if stop_event.wait(2): return
    GLib.idle_add(add_new_src)  # (1)
    log.info("Sequence ended")


t = Thread(target=timed_sequence, name="Sequence")
t.start()


ret=pipeline.set_state(Gst.State.PLAYING)

if ret==Gst.StateChangeReturn.FAILURE:
    logger.error("pipeline'da sıkıntı var")
    sys.exit(1)

bus = pipeline.get_bus()
msg = bus.timed_pop_filtered(
    Gst.CLOCK_TIME_NONE, 
    Gst.MessageType.ERROR | Gst.MessageType.EOS)

if msg:
    if msg.type == Gst.MessageType.ERROR:
        err, debug_info = msg.parse_error()
        logger.error(f"Error received from element {msg.src.get_name()}: {err.message}")
        logger.error(f"Debugging information: {debug_info if debug_info else 'none'}")

    elif msg.type == Gst.MessageType.EOS:
        logger.info("End-Of-Stream reached.")

    else:
        # This should not happen as we only asked for ERRORs and EOS
        logger.error("Unexpected message received.")

stop_event.set()
t.join()

In cb_newpad

gstname= audio/mpeg
In cb_newpad

gstname= video/x-h264
features= <Gst.CapsFeatures object at 0x7f6d0cd0cd60 (GstCapsFeatures at 0x7f6b8c003740)>


Failed to link src pad to sink pad:video/x-h264
ERROR:__main__:Error received from element sink: Output window was closed
ERROR:__main__:Debugging information: xvimagesink.c(554): gst_xv_image_sink_handle_xevents (): /GstPipeline:test-pipeline/GstXvImageSink:sink
